In [1]:
%env THEANO_FLAGS=device=gpu0,floatX=float32
import utils
from ADsnetwork import LeMoN_AI
import numpy as np

env: THEANO_FLAGS=device=gpu0,floatX=float32


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)


In [2]:
class DataProcessor(object):
    
    def __init__(self, points_amount=38):
        self.points_amount = points_amount
        
        
    def read(self, path_to_read):
        reader = c3d.Reader(open(path_to_read, 'rb'))
        coords = np.array(map(lambda x: x[1], list(reader.read_frames())))
        return coords[:,:,:3]
        
        
    def write(self, path_to_write, matrix, additional_matrix=None):
        assert np.ndim(matrix)== 3
        assert matrix.shape[2] == 3 and matrix.shape[1] == self.points_amount
        
        if additional_matrix == None:
            # define last two columns in the matrix of coordinates with zeros 
            additional_matrix = np.zeros(dtype="float32", shape=(matrix.shape[0], self.points_amount, 2))
            
        writer = c3d.Writer()
        # frames - generator of tuples (coord_matrix,  analog)
        frames = ((np.concatenate((matrix[i]*-1, additional_matrix[i]), axis=1), np.array([[]])) for i in range(len(matrix)))
        writer.add_frames(frames)
        
        with open(path_to_write, 'wb') as h:
            writer.write(h)

# Get WAV

In [3]:
C3D_FR = 480
SR = 43680
music = utils.get_sound_from_wav("", C3D_FR, SR)

IsADirectoryError: [Errno 21] Is a directory: '/home/notebook/LeMoN-AI'

In [55]:
Net = LeMoN_AI(trainable=False, prod=True)

Music conv:  (None, 256, 3)
Shift conv:  (None, 256, 10)
Position pool:  (None, 512)
Conc:  (None, 1024)
Output:  (None, 114)


In [56]:
Net.load()

In [9]:
import theano
import theano.tensor as T
import lasagne

In [10]:
input_music_var = T.tensor3("Music input")
input_shift_var = T.tensor3("Shift input")
MUSIC_SHAPE = (None, 4, 200)
START_POSITION_SHAPE = (None, 38, 3)
SHIFT_SHAPE = (None, 19, 38*3)
input_position_var = T.tensor3("Start position")
delta_mov_var = T.matrix("Delta moving")

music_input = lasagne.layers.InputLayer(shape=MUSIC_SHAPE, input_var=input_music_var)
music_input = lasagne.layers.BatchNormLayer(music_input)
music_dims = lasagne.layers.DimshuffleLayer(music_input, (0, 2, 1))
music_conv = lasagne.layers.Conv1DLayer(music_dims, 256, 2, name="Music")
music_conv = lasagne.layers.batch_norm(music_conv)
# music_pool = lasagne.layers.GlobalPoolLayer(music_conv, T.max)
print("Music conv: ", music_conv.output_shape)

shift_input = lasagne.layers.InputLayer(shape=SHIFT_SHAPE, input_var=input_shift_var)
shift_input = lasagne.layers.BatchNormLayer(shift_input)
shift_dims = lasagne.layers.DimshuffleLayer(shift_input, (0, 2, 1))
shift_conv = lasagne.layers.Conv1DLayer(shift_dims, 256, 10, name="shift_conv")
shift_conv = lasagne.layers.batch_norm(shift_conv)
# shift_pool = lasagne.layers.GlobalPoolLayer(music_conv, T.max)
print("Shift conv: ", shift_conv.output_shape)

position_input = lasagne.layers.InputLayer(shape=START_POSITION_SHAPE, input_var=input_position_var)
position_input = lasagne.layers.BatchNormLayer(position_input)
position_dims = lasagne.layers.DimshuffleLayer(position_input, (0, 2, 1))
position_conv = lasagne.layers.Conv1DLayer(position_dims, 128, 2, name="Conv0")
position_conv = lasagne.layers.batch_norm(position_conv)
position_pool = lasagne.layers.Pool1DLayer(position_conv, 2, name="Pool0")
position_conv = lasagne.layers.Conv1DLayer(position_pool, 512, 2, name="Conv1")
position_conv = lasagne.layers.batch_norm(position_conv)
position_pool = lasagne.layers.GlobalPoolLayer(position_conv, pool_function=T.max, name="Gpool")
print("Position pool: ", position_pool.output_shape)


lstm_music = lasagne.layers.LSTMLayer(music_conv, 512, only_return_final=True, name="Music LSTM")
lstm_music = lasagne.layers.batch_norm(lstm_music)
# lstm_music = lasagne.layers.ReshapeLayer(lstm_music, (-1, 1, 512))

lstm_shift = lasagne.layers.LSTMLayer(shift_conv, 512, hid_init=position_pool, only_return_final=True, name="Shift LSTM")
lstm_shift = lasagne.layers.batch_norm(lstm_shift)
# lstm_shift = lasagne.layers.ReshapeLayer(lstm_shift, (-1, 1, 512))

conc = lasagne.layers.ConcatLayer([lstm_music, lstm_shift], axis=1)
print("Conc: ", conc.output_shape)
dense_0 = lasagne.layers.DenseLayer(conc, 256, nonlinearity=lasagne.nonlinearities.linear, name="Dense1")
output = lasagne.layers.DenseLayer(dense_0, 38*3, nonlinearity=lasagne.nonlinearities.linear, name="Conv2")
print("Output: ", output.output_shape)

reshape_output = lasagne.layers.ReshapeLayer(output, (-1, 38, 3))
output = output
reshape_output = lasagne.layers.ReshapeLayer(output, (-1, 38, 3))
out = lasagne.layers.get_output(output)

predict = theano.function([input_music_var, input_shift_var,
        input_position_var], out, allow_input_downcast=True)

Music conv:  (None, 256, 3)
Shift conv:  (None, 256, 10)
Position pool:  (None, 512)
Conc:  (None, 1024)
Output:  (None, 114)


In [11]:
lasagne.layers.set_all_param_values(output, np.load("weights.npy"))

In [12]:
out = lasagne.layers.get_output(output)

predict = theano.function([input_music_var, input_shift_var,
        input_position_var], out, allow_input_downcast=True)

In [ ]:
motion = motion.reshape((-1, 38*3))

In [ ]:
mus = mus[0]

In [ ]:
mus += np.random.random(())

In [ ]:
motion = motion.reshape((-1, 38, 3))!

In [13]:
def moves_to_position(start_pos, matrix_of_movements):
    #assert np.ndim(matrix_of_movements) == 3 and start_pos.shape == (39,3)
    #assert matrix_of_movements.shape[1] == 39 and matrix_of_movements.shape[2] == 3
    
    return start_pos + np.sum(matrix_of_movements, axis=0)

In [50]:
pos, mus = utils.get_data("ok_dat.npy")

In [51]:
pos = pos[0][5900:6000]
motion = np.array(pos)

In [57]:
mt = []
for i in range(70, 38500):
    
    print(i)
    
    new_motion = Net.predict([np.load("spectr/0.{}.spec".format(i)).astype("float32")],
                             utils.positions_to_moves(motion[-20:]).reshape((1, 19, 38*3)).astype("float32"),
                             motion[i].reshape((1, 38, 3)).astype("float32"))
    motion = np.concatenate((motion, moves_to_position(motion[-1], new_motion.reshape(38, 3)).reshape(-1,38, 3)))

70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116


KeyboardInterrupt: 

In [53]:
motion.shape

(5244, 38, 3)

In [48]:
m = DataProcessor()

In [18]:
import c3d

In [54]:
m.write("ex.c3d", motion)

/anaconda3/lib/python3.5/site-packages/c3d.py:564: UserWarning: missing parameter ANALOG:LABELS
  warnings.warn('missing parameter {}'.format(name))
/anaconda3/lib/python3.5/site-packages/c3d.py:564: UserWarning: missing parameter ANALOG:DESCRIPTIONS
  warnings.warn('missing parameter {}'.format(name))


In [ ]:
utils.positions_to_moves(motion[-20:]).shape

In [ ]:
len(motion)

In [ ]:
np.load("spectr/0.{}.spec".format(i)).shape

In [ ]:
motion[len(motion)-19:-1].shape

In [ ]:
utils.positions_to_moves(motion[len(motion)-19:-1])

In [ ]:
motion.shape